<a href="https://colab.research.google.com/github/thmcheung/JSC270_HW2_2022_TingHoMarcusCheung/blob/main/part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 3

I have decided to do an entirely new analysis. The data set I am exploring is the Car 2022 dataset provided on Kaggle. This data set consists of all/most cars that are available to buy in India and their specifications. It was derived from the website https://www.cardekho.com/. The link for the dataset is https://www.kaggle.com/datasets/tr1gg3rtrash/cars-2022-dataset.

Importing the data and libraries and processing the dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf
import requests

url = 'https://drive.google.com/file/d/1TrWVojFPtf-813IKtiVWXpuuSoNV7Wyx/view?usp=sharing'
url ='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, header=None)
df.columns = ["car_name","reviews_count","fuel_type","engine_displacement","no_cylinder","seating_capacity","transmission_type",
              "fuel_tank_capacity","body_type","rating","starting_price","ending_price","max_torque_nm","max_torque_rpm","max_power_bhp","max_power_rp"]
df = df.drop(0)
print(df.info())
print(df['fuel_type'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 1 to 203
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car_name             203 non-null    object
 1   reviews_count        203 non-null    object
 2   fuel_type            203 non-null    object
 3   engine_displacement  203 non-null    object
 4   no_cylinder          203 non-null    object
 5   seating_capacity     202 non-null    object
 6   transmission_type    203 non-null    object
 7   fuel_tank_capacity   203 non-null    object
 8   body_type            203 non-null    object
 9   rating               203 non-null    object
 10  starting_price       203 non-null    object
 11  ending_price         203 non-null    object
 12  max_torque_nm        203 non-null    object
 13  max_torque_rpm       203 non-null    object
 14  max_power_bhp        203 non-null    object
 15  max_power_rp         203 non-null    object
dtypes: objec

Petrol      125
Diesel       55
Electric     16
CNG           7
Name: fuel_type, dtype: int64

The question I am interested in is predicting the max horsepower output of the car given its other important specifications (engine displacement and number of cylinders).

Notice that the data type of the columns are all objects. After checking the data type of the columns, I realized that there are no missing values, however, all of the data that were meant to be integers are represented as strings. Furthermore, I noticed that not all cars are petrol, so limiting the analysis to petrol cars would be better.

The first linear regression model I want to fit is the relationship between the engine displacement and the horsepower. I want to do this because most cars label their engine displacement in their name and the specification is a good indication of the power of the car. For example, the C63 AMG Mercedes was named after its 6.2 liter engine and it has massive horsepower.

Create new dataframe of only petrol cars and convert the strings into integers.


In [ ]:
petrol = df[df['fuel_type'] != 'Electric']
#convert horsepower into integer
petrol['engine_displacement'] = petrol['engine_displacement'].astype(int)
#convert max_power_bhp into float
petrol['max_power_bhp'] = petrol['max_power_bhp'].astype(float)

<ipython-input-12-2cb515ebbf0a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  petrol['engine_displacement'] = petrol['engine_displacement'].astype(int)
<ipython-input-12-2cb515ebbf0a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  petrol['max_power_bhp'] = petrol['max_power_bhp'].astype(float)


Find correlation between 'max_power_bhp' and 'engine_displacement'

In [ ]:
selected_columns = petrol[['max_power_bhp', 'engine_displacement']]
corr_matrix = selected_columns.corr()
print(corr_matrix)

                     max_power_bhp  engine_displacement
max_power_bhp             1.000000             0.862326
engine_displacement       0.862326             1.000000


As shown by the correlation index, the two variables have a strong positive relationship, as expected.

Fit linear regression model where the dependent varaible is 'max_power_bhp' and the independent variable is 'engine displacement'.

In [ ]:
reg1 = smf.ols('max_power_bhp ~ engine_displacement', data = petrol).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:          max_power_bhp   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     536.5
Date:                Tue, 14 Feb 2023   Prob (F-statistic):           1.43e-56
Time:                        19:53:12   Log-Likelihood:                -1122.4
No. Observations:                 187   AIC:                             2249.
Df Residuals:                     185   BIC:                             2255.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -36.4620    

As shown in regression model, the R-squared of the model is 0.744, which shows that the model is a rather fit linear model of the graph. The coefficients state that the baseline for the horsepower is -36.46 when the displacement is 0, which is impossible because no PETROL engine can have a displacement of 0. The horsepower increases by 0.1203 when there is a 1 ml increase in displacement on average.

Now, I want to find out the relationship between the number of cylinders and the horsepower of the car.

Again, I need to convert the number of cylinders into integers.

In [ ]:
petrol['no_cylinder'] = petrol['no_cylinder'].astype(int)

<ipython-input-15-a38e926373ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  petrol['no_cylinder'] = petrol['no_cylinder'].astype(int)


Now, fit the linear regression model.

In [ ]:
reg1 = smf.ols('max_power_bhp ~ no_cylinder', data = petrol).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:          max_power_bhp   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     600.2
Date:                Tue, 14 Feb 2023   Prob (F-statistic):           5.66e-60
Time:                        19:53:12   Log-Likelihood:                -1114.5
No. Observations:                 187   AIC:                             2233.
Df Residuals:                     185   BIC:                             2239.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -125.2410     17.345     -7.221      

As shown, the model is rather fit because the r squared is 0.765, which is pretty large.

Furthermore, the intercept of the horsepower is -125.82, which is again unrealistic because no PETROL car can have 0 cylinders. Also, the model shows us that an increase of 1 cylinder will lead to an increase of 76.3 horsepowers.

Lastly, I want to fit a linear regression model that uses both the number of cylinders and engine displacement as independent variables and horsepower as the dependent variable.

In [ ]:
reg1 = smf.ols('max_power_bhp ~ no_cylinder + engine_displacement', data = petrol).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:          max_power_bhp   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     324.5
Date:                Tue, 14 Feb 2023   Prob (F-statistic):           4.57e-61
Time:                        19:53:12   Log-Likelihood:                -1108.4
No. Observations:                 187   AIC:                             2223.
Df Residuals:                     184   BIC:                             2233.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            -100.9387    

As shown by the summary of the model, this model is a better model than both the other models because the R-squared is larger than both the others.

This model suggestst that a petrol car with 0 cylinders and 0 engine displacement has an average of -101.47 horsepower, which is again unrealistic. An increase of 1 cylinder will on average increase horsepower by 47.53 and the increase of the engine_displacement of 1 ml will increase horsepower by 0.049.

Now, as a fun bonus, I will use this model to predict the horsepower of my car, which is a Mazda 3 with an engine displacement of 2.5 liters and 4 cylinders.

In [ ]:
predicted_bhp = -101.4779 + 47.5265 * 4 + 0.0491 * 2500
print(predicted_bhp)

211.3781


The answer is 211.3781 hp, which is a little lower than its claimed bhp, which is 227hp, which means the model underpredicts the bhp of my car, which also means that my car has a higher power output compared with the average car in the dataset with similar engine displacement and number of cylinders.